In [1]:
import sys

lib_dir = "/home/daniele/documents/github/ftt01/phd/share/lib"
sys.path.insert( 0, lib_dir )

from lib import *
import subprocess
import psycopg2
# import datetime as dt

In [2]:
class local_args():

    def __init__(self) -> None:
        pass

    def add_start_date(self,  start_date):
        self.start_date = start_date
    
    def add_end_date(self,  end_date):
        self.end_date = end_date

    def add_variable(self,  variable):
        self.variable = variable
    
    def add_meta_grid(self,  meta_grid):
        self.meta_grid = meta_grid

    def add_output_path(self,  output_path):
        self.output_path = output_path
        mkNestedDir( output_path )

In [3]:
args = local_args()
args.add_start_date("2010-01-01T00:00:00")
args.add_end_date("2019-12-31T23:59:00")

In [4]:
start_date = dt.datetime.strptime( args.start_date, '%Y-%m-%dT%H:%M:%S' )
end_date = dt.datetime.strptime( args.end_date, '%Y-%m-%dT%H:%M:%S' )
aggregation_at = '1H'
dates = pd.date_range(start_date, end_date, freq=aggregation_at)

In [5]:
def get_postgres_connection():

    db_name = 'meteo'
    db_user = 'postgres'
    db_password = 'pgAifa2Bias?'
    db_host = '172.20.0.2'

    return psycopg2.connect(database=db_name, user=db_user, password=db_password, host=db_host)

In [6]:
def get_point( y, x, epsg=4326, tolerance=0.01 ):

    c_id = None

    sql_exist = '''
        SELECT COUNT(*)
        FROM ecmwf.era5_points
        WHERE ST_Contains(
            ST_Transform(
                ST_MakeEnvelope({min_lon}, {min_lat}, {max_lon}, {max_lat}, {epsg}), 4326 ),
            era5_points.geom)
        LIMIT 1;'''

    min_lat = y - tolerance
    max_lat = y + tolerance
    min_lon = x - tolerance
    max_lon = x + tolerance

    sql_exist = sql_exist.format(
        min_lat=min_lat,
        min_lon=min_lon,
        max_lat=max_lat,
        max_lon=max_lon,
        epsg=epsg
    )

    sql_select = '''
        SELECT ecmwf.era5_points.id
        FROM ecmwf.era5_points
        ORDER BY ecmwf.era5_points.geom <#> ST_SetSRID(ST_MakePoint({x},{y}),{epsg})
        LIMIT 1;'''.format(
            x=x,
            y=y,
            epsg=epsg
        )
    
    # print(sql_select)

    conn = get_postgres_connection()
    try:
        with conn.cursor() as cur:
            cur.execute(sql_exist)
            rows = cur.fetchall()
            if rows[0][0] != 0:
                cur.execute(sql_select)
                c_id = int(cur.fetchall()[0][0])
            else:
                print("Not existing point!")
    finally:
        conn.close()

    return c_id

In [7]:
def sql_to_dataframe(conn, query, column_names):

    # print(query)
    cursor = conn.cursor()
    try:
        cursor.execute(query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    # The execute returns a list of tuples:
    tuples_list = cursor.fetchall()
    cursor.close()
    # Now we need to transform the list into a pandas DataFrame:
    df = pd.DataFrame(tuples_list, columns=column_names)
    return df

In [8]:
full_df = pd.DataFrame(index=dates)
full_df.index.name = 'datetime'

In [9]:
sql_get_data_metadata_tp = '''
SELECT datetime, value
FROM ecmwf.era5_values
WHERE datetime >= '{start_datetime}' 
    AND datetime <= '{end_datetime}' 
    AND variable = '{variable}'
    AND point = {c_id}
GROUP BY datetime, value
ORDER BY datetime
'''

In [10]:
sql_get_data_geom_tp = '''
WITH inner_points AS (
    WITH poly AS (
        SELECT ST_Buffer(geom::geography, 5000)::geometry as geom
        FROM geometries.eu_ita_regions
        WHERE name = 'Trentino-Alto Adige'
    ),
    points AS (
        SELECT id, geom
        FROM ecmwf.era5_points
    )
    SELECT points.id as id, points.geom
    FROM points, poly
    WHERE ST_Contains(poly.geom, points.geom)
)
SELECT point, datetime, value
FROM ecmwf.era5_values, inner_points
WHERE datetime >= '{start_datetime}' 
    AND datetime <= '{end_datetime}' 
    AND variable = '{variable}'
    AND point = inner_points.id
GROUP BY point, datetime, value
ORDER BY point, datetime
'''

In [11]:
sql_get_data_metadata_2t = '''
SELECT datetime, value 
FROM ecmwf.era5_values
WHERE datetime >= '{start_datetime}' 
    AND datetime <= '{end_datetime}' 
    AND variable = '{variable}'
    AND point = {c_id}
ORDER BY datetime
'''

In [12]:
sql_get_data_geom_2t = '''
WITH inner_points AS (
    WITH poly AS (
        SELECT ST_Buffer(geom::geography, 5000)::geometry as geom
        FROM geometries.eu_ita_regions
        WHERE name = 'Trentino-Alto Adige'
    ),
    points AS (
        SELECT id, geom
        FROM ecmwf.era5_points
    )
    SELECT points.id as id, points.geom
    FROM points, poly
    WHERE ST_Contains(poly.geom, points.geom)
)
SELECT point, datetime, value
FROM ecmwf.era5_values, inner_points
WHERE datetime >= '{start_datetime}' 
    AND datetime <= '{end_datetime}' 
    AND variable = '{variable}'
    AND point = inner_points.id
GROUP BY point, datetime, value
ORDER BY point, datetime
'''

In [67]:
def get_data_id(c_id):
    sql_get_data = sql_get_data_metadata_tp.format(
        start_datetime = (start_date-dt.timedelta(hours=1)).strftime('%Y-%m-%d %H:%M'),
        end_datetime = end_date.strftime('%Y-%m-%d %H:%M'),
        variable = 'tp',
        c_id = c_id
    )
    # print(sql_get_data)
    c_df = sql_to_dataframe( get_postgres_connection(), sql_get_data, column_names = ['datetime', 'value'])
    c_df.set_index('datetime', inplace=True)
    ### put the old_id or the c_id in the columns of full_df
    c_df.rename(columns={'value':c_id}, inplace=True)

    ## precipitation
    c_df = resample_timeseries( 
        c_df, res_type='sum', 
        step=aggregation_at, offset=False )
    # from meters to mm
    c_df = c_df * 1000
    c_df = c_df[start_date:end_date]

    precipitation_era5_df = pd.concat([full_df,c_df], axis=1, join='inner')
    precipitation_era5_df[c_id] = [ round(float(v),3) for v in precipitation_era5_df[c_id].values ]
    precipitation_era5_df.rename(columns={c_id:'values'}, inplace=True)

    #########################

    sql_get_data = sql_get_data_metadata_2t.format(
            start_datetime = start_date.strftime('%Y-%m-%d %H:%M'),
            end_datetime = end_date.strftime('%Y-%m-%d %H:%M'),
            variable = '2t',
            c_id = c_id
        )
    c_df = sql_to_dataframe( get_postgres_connection(), sql_get_data, column_names = ['datetime', 'value'])
    c_df.set_index('datetime', inplace=True)
    ### put the old_id or the c_id in the columns of full_df
    c_df.rename(columns={'value':c_id}, inplace=True)

    ## temperature
    c_df = resample_timeseries( 
        c_df, res_type='mean', 
        step=aggregation_at, offset=False )
    # from Kelvin to Celsius
    c_df = c_df - 273.15

    temperature_era5_df = pd.concat([full_df,c_df], axis=1, join='inner')
    temperature_era5_df[c_id] = [ round(float(v),3) for v in temperature_era5_df[c_id].values ]
    temperature_era5_df.rename(columns={c_id:'values'}, inplace=True)

    return precipitation_era5_df,temperature_era5_df

In [76]:
output_path = '/media/windows/projects/conferences/EUG23/data/era5/'
mkNestedDir( output_path + 'precipitation' )
mkNestedDir( output_path + 'temperature' )

In [77]:
def save_point(lat,lon,name,output_path):
    c_id = get_point(lat,lon)
    precipitation_era5,temperature_era5 = get_data_id(c_id)

    precipitation_era5.to_csv(output_path+'precipitation/'+name+'.csv')
    temperature_era5.to_csv(output_path+'temperature/'+name+'.csv')

In [79]:
# Senales
save_point(lat=46.57, lon=11.03, name='senales', output_path=output_path)

In [80]:
# Passirio
save_point(lat=46.82, lon=11.28, name='passirio', output_path=output_path)

In [81]:
# Val di Sole
save_point(lat=46.32, lon=10.78, name='valdisole', output_path=output_path)